In [1]:
import pandas as pd
import tensorflow as tf

2025-03-16 22:26:32.276014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742163992.324328    6661 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742163992.336921    6661 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742163992.373383    6661 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742163992.373417    6661 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742163992.373420    6661 computation_placer.cc:177] computation placer alr

In [2]:
df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',
                     infer_datetime_format=True, index_col='datetime', header=0)
df

/tmp/ipykernel_6661/513246612.py:1: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('https://drive.google.com/uc?id=1AZRfFoyekqSYpri5183RmJjciRGz_ood', sep=',',


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0
...,...,...,...,...,...,...,...
2007-02-14 17:19:00,0.636,0.140,241.16,2.6,0.0,0.0,0.0
2007-02-14 17:20:00,0.552,0.000,240.46,2.2,0.0,0.0,0.0
2007-02-14 17:21:00,0.538,0.000,239.74,2.2,0.0,0.0,0.0


In [3]:
def normalize_series(data, min, max):
    data = data - min
    data = data / max
    return data
data = df.values
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

In [5]:
N_FEATURES = len(df.columns)
N_FEATURES

7

In [6]:
SPLIT_TIME = int(len(data) * 0.5)
x_train = data[:SPLIT_TIME]
x_valid = data[SPLIT_TIME:]

In [7]:
def windowed_dataset(series, batch_size, n_past=24, n_future=24, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past + n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past + n_future))
    ds = ds.map(lambda w: (w[:n_past], w[n_past:]))
    return ds.batch(batch_size).prefetch(1)

In [8]:
BATCH_SIZE = 32
N_PAST = 24
N_FUTURE = 24
SHIFT = 1
# Kode untuk membuat windowed datasets
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE,
                                 n_past=N_PAST, n_future=N_FUTURE,
                                 shift=SHIFT)

2025-03-16 22:30:36.002588: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(N_FEATURES)
    ])

/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('mae') < 0.055 and logs.get('val_mae') < 0.055):
                self.model.stop_training = True
 
callbacks = myCallback()

In [11]:
# Kode untuk melakukan menyusun struktur sesuai dengan machine learning
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3) 
model.compile(loss='mae',
                  optimizer= optimizer,
                  metrics=["mae"])

In [12]:
model.fit(train_set,
          validation_data=(valid_set),
          epochs=100,
          callbacks=callbacks,
          verbose=1
    )

Epoch 1/100
   1347/Unknown 6s 4ms/step - loss: 0.0771 - mae: 0.0771

2025-03-16 22:33:21.180526: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/usr/local/python/3.12.1/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.0771 - mae: 0.0771 - val_loss: 0.0625 - val_mae: 0.0625
Epoch 2/100
  45/1349 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0692 - mae: 0.0692

2025-03-16 22:33:25.624837: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0615 - mae: 0.0615 - val_loss: 0.0606 - val_mae: 0.0606
Epoch 3/100
  46/1349 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.0652 - mae: 0.0652

...

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0573 - mae: 0.0573 - val_loss: 0.0562 - val_mae: 0.0562
Epoch 10/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0569 - mae: 0.0569 - val_loss: 0.0555 - val_mae: 0.0555
Epoch 11/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0567 - mae: 0.0567 - val_loss: 0.0556 - val_mae: 0.0556
Epoch 12/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0564 - mae: 0.0564 - val_loss: 0.0555 - val_mae: 0.0555
Epoch 13/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0564 - mae: 0.0564 - val_loss: 0.0557 - val_mae: 0.0557
Epoch 14/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0562 - mae: 0.0562 - val_loss: 0.0556 - val_mae: 0.0556
Epoch 15/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0561 - mae: 0.0561 - val_loss: 0.0562 - val_mae: 0.0562
Epoch 16/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0562 - mae: 0.0562 - val_loss: 0.0558 - val_mae: 0.0558
Epoch 17/100
  44/1349 ━━━━━━━━━━━━━━

2025-03-16 22:35:43.461367: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0561 - mae: 0.0561 - val_loss: 0.0558 - val_mae: 0.0558
Epoch 18/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0561 - mae: 0.0561 - val_loss: 0.0564 - val_mae: 0.0564
Epoch 19/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0561 - mae: 0.0561 - val_loss: 0.0554 - val_mae: 0.0554
Epoch 20/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0560 - mae: 0.0560 - val_loss: 0.0555 - val_mae: 0.0555
Epoch 21/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0558 - mae: 0.0558 - val_loss: 0.0555 - val_mae: 0.0555
Epoch 22/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0558 - mae: 0.0558 - val_loss: 0.0565 - val_mae: 0.0565
Epoch 23/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0559 - mae: 0.0559 - val_loss: 0.0560 - val_mae: 0.0560
Epoch 24/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0559 - mae: 0.0559 - val_loss: 0.0554 - val_mae: 0.0554
Epoch 25/100
1349/1349 ━━━━━━━━━━━━━━

2025-03-16 22:38:09.379751: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0553 - mae: 0.0553 - val_loss: 0.0548 - val_mae: 0.0548
Epoch 34/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0554 - mae: 0.0554 - val_loss: 0.0552 - val_mae: 0.0552
Epoch 35/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0554 - mae: 0.0554 - val_loss: 0.0553 - val_mae: 0.0553
Epoch 36/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0553 - mae: 0.0553 - val_loss: 0.0557 - val_mae: 0.0557
Epoch 37/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0554 - mae: 0.0554 - val_loss: 0.0554 - val_mae: 0.0554
Epoch 38/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0554 - mae: 0.0554 - val_loss: 0.0552 - val_mae: 0.0552
Epoch 39/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0554 - mae: 0.0554 - val_loss: 0.0552 - val_mae: 0.0552
Epoch 40/100
1349/1349 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 0.0552 - mae: 0.0552 - val_loss: 0.0552 - val_mae: 0.0552
Epoch 41/100
1349/1349 ━━━━━━━━━━━━━━

In [14]:
train_pred = model.predict(train_set)
train_pred[0][0]


     24/Unknown 0s 5ms/step

1349/1349 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


array([ 3.7653336e-01,  1.8744251e-01,  3.4853041e-02,  3.5812819e-01,
        1.9343803e-04, -1.7645257e-03,  8.3028764e-01], dtype=float32)